# Imports

In [1]:
import pickle as pk
import numpy as np
from collections import Counter
import itertools

# Hidden Markov Model (HMM)

In [2]:
with open("train10.pkl","rb") as file:
    train_10 = pk.load(file)

with open("test10.pkl","rb") as file:
    test_10 = pk.load(file)
    
with open("train20.pkl","rb") as file:
    train_20 = pk.load(file)

with open("test20.pkl","rb") as file:
    test_20 = pk.load(file)

In [3]:
print("len(train_10) = {}\nlen(test_10)  = {}\nlen(train_20) = {}\nlen(test_20)  = {}"\
      .format(len(train_10),len(test_10),len(train_20),len(test_20)))

len(train_10) = 29057
len(test_10)  = 1501
len(train_20) = 27184
len(test_20)  = 3374


In [4]:
class HMM:
    
    def __init__(self, train):
        
        flat = list(itertools.chain.from_iterable(train))
        
        self.state_list = np.unique([el[1] for el in flat])
        self.N = len(self.state_list) # state_list length, just to verify that all alphabet appear
        
        self.obs_list = np.unique([el[0] for el in flat])
        self.M = len(self.obs_list) # obs_list length, just to verify that all alphabet appear
        
        self.make_index(self.state_list, self.obs_list)
        
        self.init_state_proba = self.init_state_proba(train)
        
        self.emission_proba = self.emission_proba(train)
        
        self.trans_proba = self.trans_proba(train)
        
        self.trans_proba_2 = self.trans_proba_2(train)
        
    def make_index(self, state_list, obs_list):
        
        self.state_index = {}
        self.obs_index = {}
        self.index_obs = {}
        
        for i, el in enumerate(state_list):
            self.state_index[el] = i
            
        for i, el in enumerate(obs_list):
            self.obs_index[el] = i
            self.index_obs[i] = el
    
    # initial state proba
    def init_state_proba(self, train):
        
        pi = np.zeros(self.N)
        
        for el in train:
            
            pi[self.state_index[el[0][1]]] += 1
            
        return pi/np.sum(pi)
    
    # transition matrix for first order HMM
    def trans_proba(self, train):
        
        A = np.zeros((self.N,self.N))
        
        for el in train:
            
            for i in range(len(el)-1):
                
                A[self.state_index[el[i][1]]][self.state_index[el[i+1][1]]] += 1
                    
        for i in range(A.shape[0]):
            A[i] /= np.sum(A[i])
                
        return A
    
    # transition matrix for second order HMM
    def trans_proba_2(self, train):
        
        A = np.zeros((self.N**2,self.N))
        
        for el in train:
            
            for i in range(len(el) - 2):
                # we encode the N**2 values by the formula l1*26 + l2, 
                # with l1 and l2 the indices of the first and second letters  
                A[self.state_index[el[i][1]]*(len(self.state_list)) + self.state_index[el[i+1][1]]][self.state_index[el[i+2][1]]] += 1
        
        for i in range(A.shape[0]):
            if(np.sum(A[i]) != 0):
                A[i] /= np.sum(A[i])
                
        return A
    
    # emission matrix
    def emission_proba(self, train):
        
        B = np.zeros((self.N,self.M))
        
        flat = list(itertools.chain.from_iterable(train))
        
        count = Counter(flat)
        
        for x in self.state_list:
            for y in self.obs_list:
                B[self.state_index[x]][self.obs_index[y]] = count[(x,y)]
                
        for i in range(self.N):
            B[i] /= np.sum(B[i])
            
        return B

## Viterbi for first order HMM

In [5]:
def Viterbi(hmm,obs):
    
    # number of states
    N = hmm.N
    
    # transition matrix
    A = hmm.trans_proba
    
    # emission matrix
    B = hmm.emission_proba
    
    # intial state proba
    pi = hmm.init_state_proba
    
    state_index = hmm.state_index
    
    obs_index = hmm.obs_index
    
    index_obs = hmm.index_obs
    
    # matrix to reconstruct the most probable sequence
    most_prob_seq = np.zeros((N,len(obs)))
    
    # fill the first column with values from 0 to N-1, not required just for the semantic 
    most_prob_seq[:,0] = np.arange(N)
    
    # calculate mu_1
    mu = B[:,obs_index[obs[0]]]*pi
    
    
    for k,i in enumerate(obs[1:]):
        
        # temporary variable to hold the values of mu
        tmp = np.zeros(N)
        
        for j in hmm.state_list:
            
            vect = B[state_index[j],obs_index[i]]*A[:,state_index[j]]*mu
            
            tmp[state_index[j]] = np.max(vect)
            
            most_prob_seq[state_index[j]][k+1] = np.argmax(vect)
            
        mu = tmp
        
    seq = []
    
    seq.append(index_obs[np.argmax(mu)])
    
    # reconstruct the inverted path
    for i in range(len(obs)-1,0,-1):
    
        seq.append(index_obs[most_prob_seq[obs_index[seq[len(obs) - 1 - i]],i]])
        
    # return the most probable sequence i.e. the non-inverted path 
    return seq[::-1]

# Viterbi for second order HMM

In [6]:
def Viterbi_2(hmm,obs):
    
    if(len(obs) <= 2 ):
        return Viterbi(hmm,obs)
    # number of states
    N = hmm.N
    
    # transition matrix for second order HMM
    A = hmm.trans_proba_2

    
    # emission matrix
    B = hmm.emission_proba
    
    # intial state proba
    pi = hmm.init_state_proba
    
    state_index = hmm.state_index
    
    obs_index = hmm.obs_index
    
    index_obs = hmm.index_obs
    
    
    # list of matrices to reconstruct the most probable sequence
    c = []
        
    # calculate d_1
    d1 = B[:,obs_index[obs[0]]]*pi
    
    # calculate d_2
    
    d = d1[:,None]*hmm.trans_proba*B[:,obs_index[obs[1]]]
    
    toN = np.arange(N)
    
    for i in obs[2:]:
        
        # temporary variable to hold the values of mu
        tmp_d = np.zeros((N,N))
        tmp_c = np.zeros((N,N))
        
        for m,_ in enumerate(hmm.state_list):
            
            for n,_ in enumerate(hmm.state_list):
                
                vect = d[:,m]*A[toN*N + m, n]
                
                tmp_d[m,n] = np.max(vect)*B[n,obs_index[i]]
                
                tmp_c[m,n] = np.argmax(vect)
        
        d = tmp_d
        
        c.append(tmp_c)
    
    k = len(obs)
    seq = np.zeros(k, dtype = np.int8)
    
    x,y = np.unravel_index(d.argmax(), d.shape)
    
    seq[k-1] = y
    seq[k-2] = x
    
    for i,j in zip(range(k-3,-1,-1),range(len(c)-1,-1,-1)):
        
        seq[i] = c[j][seq[i+1],seq[i+2]]
    
    output = []
    for i in range(len(seq)):
        output.append(index_obs[seq[i]])
    
    return output

# Evaluation

In [7]:
def eval(true,test,pred):
    
    pos = 0
    
    neg = 0
    
    l1 = list(itertools.chain.from_iterable(true))
    
    l2 = list(itertools.chain.from_iterable(test))
    
    l3 = list(itertools.chain.from_iterable(pred))
    
    for i in range(len(l1)):
        
        if(l1[i] != l2[i] and l1[i] == l3[i]):
            pos +=1
            
        if(l1[i] == l2[i] and l1[i] != l3[i]):
            neg +=1
            
    return pos,neg,(np.array(l1) != np.array(l2)).sum()

def accuracy(true,pred):
    
    l1 = np.array(list(itertools.chain.from_iterable(true)))
        
    l2 = np.array(list(itertools.chain.from_iterable(pred)))
    
    return (l1 == l2).sum()/l1.shape[0]

In [8]:
def results(training,testing,f,first_order = True):
    
    hmm = HMM(training)
    test = []
    true = []
    for l in testing:
        test.append([el[0] for el in l])
        true.append([el[1] for el in l])

    pred = []

    for el in test:
        pred.append(f(hmm,el))
        
    pos,neg,over = eval(true,test,pred)
    maxlen = len(list(itertools.chain.from_iterable(true)))

    if(first_order):
        print("First order HMM made {} mistakes over {} and corrected {} mistakes over {} characters".format(neg,maxlen-over,pos,over))
    else:
        print("Second order HMM made {} mistakes over {} and corrected {} mistakes over {} characters".format(neg,maxlen-over,pos,over))
    
    print("The overall accuracy is about {0:.2f}%".format(accuracy(true,pred)*100))
    
    print("Without doing anything, the accuracy is about {0:.2f}%".format(accuracy(true,test)*100))

__Testing Viterbi for first order HMM with 10% error__

In [9]:
results(train_10,test_10,Viterbi)

First order HMM made 79 mistakes over 6575 and corrected 247 mistakes over 745 characters
The overall accuracy is about 92.12%
Without doing anything, the accuracy is about 89.82%


__Testing Viterbi for second order HMM with 10% error__

In [10]:
results(train_10,test_10,Viterbi_2,False)

Second order HMM made 129 mistakes over 6575 and corrected 448 mistakes over 745 characters
The overall accuracy is about 94.18%
Without doing anything, the accuracy is about 89.82%


__Testing Viterbi for first order HMM with 20% error__

In [11]:
results(train_20,test_20,Viterbi)

First order HMM made 363 mistakes over 13452 and corrected 1080 mistakes over 3239 characters
The overall accuracy is about 84.89%
Without doing anything, the accuracy is about 80.59%


__Testing Viterbi for second order HMM with 20% error__

In [12]:
results(train_20,test_20,Viterbi_2,False)

Second order HMM made 524 mistakes over 13452 and corrected 1919 mistakes over 3239 characters
The overall accuracy is about 88.95%
Without doing anything, the accuracy is about 80.59%
